In [ ]:
# ===========================================================
# 2. Modelado Espaciotemporal – Embeddings GNN + Agregación Departamental – SABER 11 (2015–2022)
# Autor: John Jairo Prado Piñeres
# Fecha: 2025
#
# Descripción:
# Este módulo implementa la Capa 1 y Capa 2 del Modelo Espaciotemporal Unificado
# para la predicción del rendimiento académico Saber 11 en Colombia.
#
# CAPA 1: Modelado GNN Escolar (GraphSAGE)
# - Construye el grafo escolar por año (2015–2022) utilizando k-Nearest Neighbors (kNN)
#   sobre 23 variables académicas, familiares, demográficas e institucionales.
# - Genera embeddings densos por colegio mediante un modelo GraphSAGE (64 dimensiones).
#
# CAPA 2: Agregación Departamental de Embeddings
# - Utiliza la variable territorial oficial ESTU_DEPTO_PRESENTACION para identificar
#   la unidad geográfica del desempeño Saber 11.
# - Agrega los embeddings escolares por departamento mediante promedio vectorial.
# - Produce una matriz de embeddings departamentales por año (33 departamentos × 64 dimensiones).
#
# Entradas:
# - df_preGrafo_20XX.csv: dataset por año completamente imputado, codificado y escalado
#   con variables educativas, familiares, institucionales y territoriales.
# - FEATURE_COLUMNS: lista oficial de variables usadas como features del grafo escolar.
# - TERRITORIAL_COLUMN = "ESTU_DEPTO_PRESENTACION": identificador departamental.
#
# Salidas:
# - embeddings_departamento_20XX.npy: embedding departamental para cada año.
# - tensor_temporal_2015_2022.npy: tensor espaciotemporal (8 años × 33 departamentos × 64 features)
#   listo para alimentar modelos temporales (T-GCN, EvolveGCN, Transformers).
#
# Este módulo NO modifica ni reconstruye los grafos originales; solo extiende el pipeline
# hacia el modelado espaciotemporal, preparando la entrada para la Capa 3 (modelos temporales).
# ===========================================================


# **Imports y configuración**

In [1]:
# IMPORTACIONES GENERALES

import os                              # Manejo del sistema operativo
import random                          # Generación de números aleatorios
import warnings                        # Control de advertencias
import unicodedata                     # Normalización de texto
import numpy as np                     # Computación numérica
import pandas as pd                    # Manipulación de datos

# VISUALIZACIÓN

import matplotlib.pyplot as plt        # Gráficos básicos
import seaborn as sns                  # Visualización estadística
import plotly.graph_objects as go      # Gráficos interactivos
from pyvis.network import Network      # Visualización de grafos interactivos
from IPython.display import IFrame     # Mostrar contenido HTML/IFrame

# GRAFOS CON NETWORKX

import networkx as nx                  # Construcción y análisis de grafos

# MACHINE LEARNING / PREPROCESAMIENTO (SCIKIT-LEARN)

from sklearn.preprocessing import (
    LabelEncoder,                     # Codificación de etiquetas
    MinMaxScaler,                     # Escalamiento Min–Max
    StandardScaler                    # Escalamiento estándar (Z-score)
)

from sklearn.model_selection import train_test_split     # División train/test
from sklearn.decomposition import TruncatedSVD           # Reducción de dimensionalidad
from sklearn.neighbors import NearestNeighbors           # Vecinos más cercanos
from sklearn.metrics import (                            # Métricas de regresión
    mean_absolute_error,
    mean_squared_error,
    r2_score,
    median_absolute_error
)

from sklearn.compose import ColumnTransformer            # Transformaciones por columna
from sklearn.preprocessing import OneHotEncoder          # One-hot encoding

from sklearn.neighbors import kneighbors_graph           # Grafo kNN

# PYTORCH

import torch                          # Núcleo de PyTorch
import torch.nn as nn                 # Capas y modelos
import torch.nn.functional as F       # Funciones activación/ops NN
from torch.optim import Adam          # Optimizador Adam
from torch.optim.lr_scheduler import CosineAnnealingLR   # Scheduler Coseno

# PYTORCH GEOMETRIC (GNN)

from torch_geometric.data import Data         # Objeto principal de grafos
from torch_geometric.loader import DataLoader # Cargador de grafos

from torch_geometric.nn import (              # Capas para GNN
    GCNConv,
    SAGEConv,
    GATConv,
    GINConv
)

from torch_geometric.utils import (           # Utilidades para grafos
    from_networkx,
    add_self_loops,
    to_networkx
)

# CONFIGURACIÓN DE REPRODUCIBILIDAD

def set_seed(seed=5477976):
    random.seed(seed)                           # Semilla para Python
    os.environ["PYTHONHASHSEED"] = str(seed)    # Semilla hash
    np.random.seed(seed)                        # Semilla NumPy

    torch.manual_seed(seed)                     # Semilla PyTorch CPU
    if torch.cuda.is_available():               # Semilla PyTorch GPU
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)

    # Configuración estricta de reproducibilidad
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(True, warn_only=True)

    print(f"Semilla de reproducibilidad establecida: {seed}")

# Establecer semilla global
set_seed(5477976)

Semilla de reproducibilidad establecida: 5477976


In [ ]:

# Rutas

ruta_imp = r"C:/Users/john/Desktop/Saber_11_2025/data/4_modelado"
ruta_limpios = r"C:/Users/john/Desktop/Saber_11_2025/data/2_sinDuplicados"
ruta_salida = r"C:/Users/john/Desktop/Saber_11_2025/data/4_modelado/completados"

os.makedirs(ruta_salida, exist_ok=True)

archivos = {
    2021: ("imp2021_db1.csv", "saber11_2021_limpio.csv"),
    2022: ("imp2022_db1.csv", "saber11_2022_limpio.csv"),
}

# Variables obligatorias para todos los años

vars_finales = [
    "FAMI_EDUCACIONMADRE","FAMI_EDUCACIONPADRE","FAMI_PERSONASHOGAR",
    "FAMI_CUARTOSHOGAR","FAMI_TIENEAUTOMOVIL","FAMI_TIENECOMPUTADOR",
    "FAMI_TIENEINTERNET","FAMI_TIENELAVADORA","FAMI_ESTRATOVIVIENDA",

    "PUNT_INGLES","PUNT_MATEMATICAS","PUNT_SOCIALES_CIUDADANAS",
    "PUNT_C_NATURALES","PUNT_LECTURA_CRITICA","PUNT_GLOBAL",

    "PERIODO",

    "COLE_COD_DANE_ESTABLECIMIENTO","COLE_COD_DANE_SEDE",

    "DESEMP_INGLES",

    "COLE_AREA_UBICACION","COLE_DEPTO_UBICACION",
    "COLE_MCPIO_UBICACION","COLE_NATURALEZA","COLE_JORNADA",

    "ESTU_DEPTO_PRESENTACION","ESTU_MCPIO_PRESENTACION","ESTU_GENERO"
]

# Reconstrucción por año

for year, (imp_file, limpio_file) in archivos.items():

    print(f"\n AÑO {year} \n")

    df_imp = pd.read_csv(os.path.join(ruta_imp, imp_file), low_memory=False)
    df_limpio = pd.read_csv(os.path.join(ruta_limpios, limpio_file), low_memory=False)

    # asegurar tipos
    df_imp["COLE_COD_DANE_ESTABLECIMIENTO"] = df_imp["COLE_COD_DANE_ESTABLECIMIENTO"].astype(int)
    df_limpio["COLE_COD_DANE_ESTABLECIMIENTO"] = df_limpio["COLE_COD_DANE_ESTABLECIMIENTO"].astype(int)

    # MERGE con 2 columnas clave
    df_merge = df_imp.merge(
        df_limpio[["COLE_COD_DANE_ESTABLECIMIENTO", "ESTU_GENERO"] + vars_finales],
        on=["COLE_COD_DANE_ESTABLECIMIENTO", "ESTU_GENERO"],
        how="left"
    )

    # Reporte de columnas faltantes
    for col in vars_finales:
        missing = df_merge[col].isna().sum()
        print(f"{col}: faltantes -> {missing}")

    # Seleccionar solo las columnas finales (y en orden)
    df_final = df_merge[vars_finales]

    # Guardar
    out = os.path.join(ruta_salida, f"imp{year}_db1_completo.csv")
    df_final.to_csv(out, index=False, encoding="utf-8")

    print(f"\nArchivo reconstruido: {out}\n")


# **Cargar el dataset UNIFICADO 2015–2022**

In [5]:
# Carpeta donde están los DB imputados
base_path = r"C:/Users/john/Desktop/Saber_11_2025/data/4_modelado"

# Archivos por año
files = {
    2015: "imp2015_db1.csv",
    2016: "imp2016_db1.csv",
    2017: "imp2017_db1.csv",
    2018: "imp2018_db1.csv",
    2019: "imp2019_db1.csv",
    2020: "imp2020_db1.csv",
    2021: "imp2021_db1.csv",
    2022: "imp2022_db1.csv"
}

cols_year = {}

for year, fname in files.items():
    df = pd.read_csv(os.path.join(base_path, fname), low_memory=False)
    cols_year[year] = df.columns.tolist()

# Mostrar columnas por año
for year, cols in cols_year.items():
    print(f"Año {year} → {len(cols)} columnas")
    print(cols)
    print("-" * 80)


Año 2015 → 32 columnas
['FAMI_EDUCACIONMADRE', 'FAMI_EDUCACIONPADRE', 'FAMI_PERSONASHOGAR', 'FAMI_CUARTOSHOGAR', 'FAMI_ESTRATOVIVIENDA', 'COLE_BILINGUE', 'ESTU_GENERO', 'FAMI_TIENEAUTOMOVIL', 'FAMI_TIENECOMPUTADOR', 'FAMI_TIENEINTERNET', 'FAMI_TIENELAVADORA', 'PUNT_INGLES', 'PUNT_MATEMATICAS', 'PUNT_SOCIALES_CIUDADANAS', 'PUNT_C_NATURALES', 'PUNT_LECTURA_CRITICA', 'PUNT_GLOBAL', 'PERIODO', 'ANIO', 'COLE_COD_DANE_ESTABLECIMIENTO', 'COLE_COD_DANE_SEDE', 'COLE_SEDE_PRINCIPAL', 'ESTU_PRIVADO_LIBERTAD', 'DESEMP_INGLES', 'COLE_AREA_UBICACION', 'COLE_DEPTO_UBICACION', 'COLE_MCPIO_UBICACION', 'COLE_NATURALEZA', 'COLE_GENERO', 'COLE_JORNADA', 'ESTU_DEPTO_PRESENTACION', 'ESTU_MCPIO_PRESENTACION']
--------------------------------------------------------------------------------
Año 2016 → 32 columnas
['FAMI_TIENEAUTOMOVIL', 'FAMI_TIENECOMPUTADOR', 'FAMI_TIENEINTERNET', 'FAMI_TIENELAVADORA', 'FAMI_ESTRATOVIVIENDA', 'FAMI_EDUCACIONMADRE', 'FAMI_EDUCACIONPADRE', 'FAMI_PERSONASHOGAR', 'COLE_BILINGUE',

# **COLUMNA FINALES (LISTA OFICIAL DE 26)**

In [3]:
variables_finales = [

    # --- Variables del grafo ---
    "COLE_COD_DANE_ESTABLECIMIENTO",
    "COLE_DEPTO_UBICACION",
    "COLE_MCPIO_UBICACION",

    # --- Académicas ---
    "PUNT_INGLES","PUNT_MATEMATICAS","PUNT_SOCIALES_CIUDADANAS",
    "PUNT_C_NATURALES","PUNT_LECTURA_CRITICA","PUNT_GLOBAL",
    "DESEMP_INGLES",

    # --- Familiares ---
    "FAMI_EDUCACIONMADRE","FAMI_EDUCACIONPADRE","FAMI_PERSONASHOGAR",
    "FAMI_CUARTOSHOGAR","FAMI_ESTRATOVIVIENDA","FAMI_TIENEAUTOMOVIL",
    "FAMI_TIENECOMPUTADOR","FAMI_TIENEINTERNET","FAMI_TIENELAVADORA",

    # --- Demográfica ---
    "ESTU_GENERO",

    # --- Institucionales ---
    "COLE_BILINGUE","COLE_NATURALEZA","COLE_GENERO","COLE_JORNADA"
]

df = df[variables_finales].copy()
print("Variables seleccionadas:", df.shape)


Variables seleccionadas: (568244, 24)


# **CLASIFICACIÓN DE VARIABLES POR TIPO**

In [9]:
# Numéricas puras
num_cols = [
    "PUNT_INGLES","PUNT_MATEMATICAS","PUNT_SOCIALES_CIUDADANAS",
    "PUNT_C_NATURALES","PUNT_LECTURA_CRITICA","PUNT_GLOBAL"
]

# Ordinales (label encoding + zscore)
ord_cols = [
    "DESEMP_INGLES","FAMI_EDUCACIONMADRE","FAMI_EDUCACIONPADRE",
    "FAMI_PERSONASHOGAR","FAMI_CUARTOSHOGAR","FAMI_ESTRATOVIVIENDA",
    "FAMI_TIENEAUTOMOVIL","FAMI_TIENECOMPUTADOR",
    "FAMI_TIENEINTERNET","FAMI_TIENELAVADORA",
    "ESTU_GENERO","COLE_NATURALEZA","COLE_JORNADA"
]

# OneHot
onehot_cols = ["COLE_GENERO"]

# Variables del grafo (NO ESCALAR)
vars_grafo = [
    "COLE_COD_DANE_ESTABLECIMIENTO",
    "COLE_DEPTO_UBICACION",
    "COLE_MCPIO_UBICACION"
]

# Variables no escalables
no_scale_cols = vars_grafo + ["COLE_BILINGUE"]

# **LABEL ENCODING A LAS ORDINALES**

In [10]:
label_encoders = {}

for col in ord_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))
    label_encoders[col] = le


# **ONE HOT PARA COLE_GENERO**

In [11]:

df["COLE_GENERO"] = df["COLE_GENERO"].astype(str)

# **PREPROCESSOR COMPLETO**

In [12]:

preprocessor = ColumnTransformer(
    transformers=[
        ("num_scale", StandardScaler(), num_cols),
        ("ord_scale", StandardScaler(), ord_cols),
        ("onehot", OneHotEncoder(sparse_output=False), onehot_cols),
        ("passthrough", "passthrough", no_scale_cols)
    ]
)

# **TRANSFORMAR**

In [13]:
mat = preprocessor.fit_transform(df)


# **NOMBRES FINALES DE VARIABLES**

In [14]:

onehot_names = preprocessor.named_transformers_["onehot"].get_feature_names_out(onehot_cols)

final_columns = num_cols + ord_cols + list(onehot_names) + no_scale_cols

# **CREAR df_preGrafo**

In [15]:

df_preGrafo_2015 = pd.DataFrame(mat, columns=final_columns)
print("df_preGrafo_2015:", df_preGrafo_2015.shape)

df_preGrafo_2015: (568244, 26)


# **GUARDAR CSV preGrafo**

In [16]:

df_preGrafo_2015.to_csv(
    r"C:/Users/john/Desktop/Saber_11_2025/data/3_DB_imp/imp2015_db1_preGrafo.csv",
    index=False
)
print("Archivo preGrafo guardado.")

Archivo preGrafo guardado.


# **CARGAR de nuevo (para el grafo)**

In [17]:

df_preGrafo_2015 = pd.read_csv(
    r"C:/Users/john/Desktop/Saber_11_2025/data/3_DB_imp/imp2015_db1_preGrafo.csv"
)

# **CONVERTIR VARIABLES DEL GRAFO A CÓDIGOS NUMÉRICOS**

In [18]:

for col in vars_grafo:
    df_preGrafo_2015[col] = df_preGrafo_2015[col].astype("category").cat.codes

# **MATRIZ X_grafo**

In [19]:

X_grafo = df_preGrafo_2015[vars_grafo].values
print("X_grafo:", X_grafo.shape)

X_grafo: (568244, 3)


# **CREAR GRAFO KNN (cosine k=8)**

In [20]:

knn_graph = kneighbors_graph(
    X_grafo,
    n_neighbors=8,
    metric="cosine",
    include_self=False,
    n_jobs=-1
)

KeyboardInterrupt: 

# **GENERAR edge_index**

In [ ]:

row, col = knn_graph.nonzero()
edge_index_np = np.vstack((row, col))
edge_index = torch.tensor(edge_index_np, dtype=torch.long)

print("edge_index:", edge_index.shape)

# **GUARDAR edge_index**

In [ ]:

np.save(r"C:/Users/john/Desktop/Saber_11_2025/data/4_modelado/edge_index_2015.npy", edge_index_np)
torch.save(edge_index, r"C:/Users/john/Desktop/Saber_11_2025/data/4_modelado/edge_index_2015.pt")

# **GUARDAR GRAFO COMPLETO (optional)**

In [ ]:

data_2015 = Data(
    x=torch.tensor(df_preGrafo_2015.drop(vars_grafo, axis=1).values, dtype=torch.float),
    edge_index=edge_index,
    y=torch.tensor(df_preGrafo_2015["PUNT_GLOBAL"].values, dtype=torch.float)
)

torch.save(data_2015, r"C:/Users/john/Desktop/Saber_11_2025/data/4_modelado/grafo_2015.pt")
print("Grafo guardado.")

# **Definir GraphSAGE**

In [ ]:
class GraphSAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels=64):
        super().__init__()
        self.conv1 = SAGEConv(in_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, hidden_channels)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return x


# **Construcción del grafo escolar**

In [ ]:
def construir_grafo_escolar(df, k=8):
    X = df[FEATURE_COLUMNS].values.astype(np.float32)

    knn_graph = kneighbors_graph(
        X, 
        n_neighbors=k,
        mode="connectivity",
        include_self=False,
        n_jobs=-1
    )

    edge_index = np.vstack(knn_graph.nonzero())

    data = Data(
        x=torch.tensor(X, dtype=torch.float32),
        edge_index=torch.tensor(edge_index, dtype=torch.long)
    )
    return data


# **Entrenamiento GraphSAGE por año**

In [ ]:
def entrenar_graphsage(df, anio):
    print(f"Entrenando GraphSAGE para {anio}...")

    data = construir_grafo_escolar(df)

    model = GraphSAGE(in_channels=data.x.shape[1], hidden_channels=64)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(1, 51):
        model.train()
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)
        loss = out.norm(dim=1).mean()
        loss.backward()
        optimizer.step()

    print(f"Modelo {anio} entrenado.")
    return model, data


# **Obtener embeddings por colegio**

In [ ]:
def obtener_embeddings(model, data):
    model.eval()
    with torch.no_grad():
        emb = model(data.x, data.edge_index)
    return emb.cpu().numpy()


# **Agregación de embeddings por departamento**

In [ ]:
def embedding_por_departamento(df, embeddings):
    df_emb = pd.DataFrame(embeddings)
    df_emb["departamento"] = df[TERRITORIAL_COLUMN].values

    emb_depto = df_emb.groupby("departamento").mean()
    emb_depto = emb_depto.sort_index()

    return emb_depto.values   # (33 × 64)


# **Procesar un año completo**

In [ ]:
def procesar_anio(df, anio):
    model, data = entrenar_graphsage(df, anio)

    emb = obtener_embeddings(model, data)

    emb_depto = embedding_por_departamento(df, emb)

    np.save(f"embeddings_departamento_{anio}.npy", emb_depto)

    print(f"Embedding departamental guardado: embeddings_departamento_{anio}.npy")
    return emb_depto


# **Ejecutar para todos los años 2015–2022**

In [ ]:
embeddings_all_years = []

for anio in range(2015, 2023):
    print(f"Procesando año {anio}")
    
    df = pd.read_csv(f"C:/Users/john/Desktop/Saber_11_2025/data/3_DB_imp/imp{anio}_db1_preGrafo.csv")

    assert TERRITORIAL_COLUMN in df.columns, f"ERROR: Falta {TERRITORIAL_COLUMN}"

    emb_depto = procesar_anio(df, anio)
    embeddings_all_years.append(emb_depto)

# tensor final (8 × 33 × 64)
X_temporal = np.stack(embeddings_all_years)
np.save("tensor_temporal_2015_2022.npy", X_temporal)

print("\nTensor temporal generado:")
print("Forma:", X_temporal.shape)
